Convolutional Neural Networks (LeNet)

6.6.1 LeNet Pada tingkat tinggi, (LeNet (LeNet-5) terdiri dari dua bagian: (i) encoder konvolusi yang terdiri dari dua lapisan konvolusi, dan (ii) blok padat yang terdiri dari tiga lapisan yang terhubung penuh); Arsitekturnya diringkas dalam :numref:img_lenet.

In [8]:
!pip install d2l

In [10]:
import torch
from torch import nn
from d2l import torch as d2l

net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10))

Dengan melewatkan gambar saluran tunggal (hitam putih) melalui jaringan dan mencetak bentuk keluaran pada setiap lapisan, kita dapat [memeriksa model] untuk memastikan bahwa operasinya sesuai dengan apa yang kita harapkan dari :numref:img_lenet_vert.

In [11]:
X = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape: \t',X.shape)

Conv2d output shape: 	 torch.Size([1, 6, 28, 28])
Sigmoid output shape: 	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape: 	 torch.Size([1, 6, 14, 14])
Conv2d output shape: 	 torch.Size([1, 16, 10, 10])
Sigmoid output shape: 	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape: 	 torch.Size([1, 16, 5, 5])
Flatten output shape: 	 torch.Size([1, 400])
Linear output shape: 	 torch.Size([1, 120])
Sigmoid output shape: 	 torch.Size([1, 120])
Linear output shape: 	 torch.Size([1, 84])
Sigmoid output shape: 	 torch.Size([1, 84])
Linear output shape: 	 torch.Size([1, 10])


Training

Sekarang kita telah mengimplementasikan modelnya, mari kita [menjalankan eksperimen untuk melihat bagaimana tarif LeNet di Fashion-MNIST].

In [12]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Sementara CNN memiliki lebih sedikit parameter, mereka masih bisa lebih mahal untuk dihitung daripada MLP yang sama dalam karena setiap parameter berpartisipasi dalam lebih banyak perkalian. Jika Anda memiliki akses ke GPU, ini mungkin saat yang tepat untuk menerapkannya guna mempercepat pelatihan.

Untuk evaluasi, kita perlu [melakukan sedikit modifikasi pada fungsi evaluation_accuracy] yang telah kita jelaskan di :numref:sec_softmax_scratch. Karena dataset lengkap ada di memori utama, kita perlu menyalinnya ke memori GPU sebelum model menggunakan GPU untuk menghitung dengan dataset.

In [5]:
def evaluate_accuracy_gpu(net, data_iter, device=None):
    """Compute the accuracy for a model on a dataset using a GPU."""
    if isinstance(net, nn.Module):
        net.eval()  # Set the model to evaluation mode
        if not device:
            device = next(iter(net.parameters())).device
    # No. of correct predictions, no. of predictions
    metric = d2l.Accumulator(2)

    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
                # Required for BERT Fine-tuning (to be covered later)
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

Fungsi pelatihan train_ch6 juga mirip dengan train_ch3 yang didefinisikan dalam :numref:sec_softmax_scratch. Karena kami akan mengimplementasikan jaringan dengan banyak lapisan ke depannya, kami akan mengandalkan API tingkat tinggi. Fungsi pelatihan berikut mengasumsikan model yang dibuat dari API tingkat tinggi sebagai input dan dioptimalkan sesuai dengan itu. Kami menginisialisasi parameter model pada perangkat yang ditunjukkan oleh argumen perangkat, menggunakan inisialisasi Xavier seperti yang diperkenalkan di :numref:subsec_xavier. Sama seperti MLP, fungsi kerugian kami adalah entropi silang, dan kami meminimalkannya melalui penurunan gradien stokastik minibatch. Karena setiap epoch membutuhkan waktu puluhan detik untuk berjalan, kami memvisualisasikan kehilangan pelatihan lebih sering.

In [13]:
def train_ch6(net, train_iter, test_iter, num_epochs, lr, device):
    """Train a model with a GPU (defined in Chapter 6)."""
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
                            legend=['train loss', 'train acc', 'test acc'])
    timer, num_batches = d2l.Timer(), len(train_iter)
    for epoch in range(num_epochs):
        # Sum of training loss, sum of training accuracy, no. of examples
        metric = d2l.Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (train_l, train_acc, None))
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
          f'on {str(device)}')

In [15]:
lr, num_epochs = 0.9, 10
train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

KeyboardInterrupt: ignored

ImportError: ignored

<Figure size 252x180 with 1 Axes>